# Movie Bot prototyp

Should help find movies in a conversational way.
Output Movie Recommendation and info howto stream.

## Steps

- [X] Get Movie Recommendations via LLM and userinput
- [X]  Use LLM function call to get the titles
- [X]  search movies via TMDB Api and find streaming provider **AND AALL OTHER MOVIE_INFO DATA (PLAYTIME; ACTORS; GENRES)**
- [X]  write recommendation to chat output with context form 3

### Bonus Tasks
- [ ] find newer once via tmdb api and recommendet titles
- [X] if user asks further questions answer based on api call
- [ ] if user asks for new recros do this! - do not recro same movies in one session
- [ ] build website with chat interface
- [ ] build website with images and direct links
- [ ] test with other cheaper llms
- [ ] filter for providers and pref to rent / flatrate
- [ ] filter this by user input
- [ ] use block/watch list / streaming source and so on


In [1]:
import os

#global sessionvars
blacklists=[]

# Setup keys
openai_api_key = os.environ['openai_api_key']
tmdb_bearer = os.environ['tmdb_bearer']


In [ ]:
# Setup env
#!pip install -qU langchain openai


In [2]:
#tmdb api
# moviedb tool singe title search
import requests
import json
from difflib import get_close_matches


headers = {
    "accept": "application/json",
    "Authorization": tmdb_bearer
}

def get_basic_data_from_tmdb_for_titles(titles: list[str]):
  print(f"get_basic_data_from_tmdb_for_titles",titles)
  movies =[]
  for title in titles:
    movie = get_basic_data_from_tmdb_for_title(title)
    if movie:
      movies.append(movie)
  fulldata_as_string = create_data_list(movies)
  return fulldata_as_string


def get_detail_data_from_tmdb_for_titles(titles: list[str]):
  print(f"get_detail_data_from_tmdb_for_titles",titles)
  movies =[]
  for title in titles:
    movie = get_detail_data_from_tmdb_for_title(title)
    if movie:
      movies.append(movie)
  return movies

def get_basic_data_from_tmdb_for_title(title: str):
  try:
    print(f"get_movie_data_from_tmdb","title=",title)
    fulldata = create_basic_movie_data(title)
    return fulldata
  except:
     return None

def get_detail_data_from_tmdb_for_title(title: str):
  try:
    print(f"get_movie_data_from_tmdb","title=",title)
    fulldata = create_movie_data(title)
    return fulldata
  except:
     return None

def find_movie_basic(title, lang):

  title = title.replace(' ', '+')
  url = "https://api.themoviedb.org/3/search/movie?include_adult=false&language="+lang+"&page=1&query="+ title

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    data = json.loads(response.text)
    print(f"find_movie for", title, "response: ",data)

    if "results" in str(data) and len(data["results"]) > 0:
      print(f"movie found!")
      movie = get_movie_form_search(data["results"], title)
      return movie

  return None

def find_movie(title, lang):

  title = title.replace(' ', '+')
  url = "https://api.themoviedb.org/3/search/movie?include_adult=false&language="+lang+"&page=1&query="+ title

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    data = json.loads(response.text)
    print(f"find_movie for", title, "response: ",data)

    if "results" in str(data) and len(data["results"]) > 0:
      print(f"movie found!")
      movie = get_movie_form_search(data["results"], title)
      movie = get_detail_moviedata(get_movie_id(movie))
      movie = json.loads(movie)
      return movie

  return None


def get_movie_id(movie):
  print(f"get_movie_id for ", movie)
  if movie:
      return movie["id"]


def get_watch_providers(id):
  #print(id)
  url = "https://api.themoviedb.org/3/movie/"+str(id)+"/watch/providers"
  response = requests.get(url, headers=headers)
  return response.text

def get_detail_moviedata(id):
  url = "https://api.themoviedb.org/3/movie/"+str(id)+"?language=en-US"
  response = requests.get(url, headers=headers)
  return response.text


def filter_watch_providers(data, lang):
  # Filter data under "<lang>"
  data = json.loads(data)
  if lang in data["results"]:
    filtered_data = data["results"][lang]
    # Print the filtered JSON
    return filtered_data



def get_watch_providers_via_subtype(data, subtype):
#subtype = "bye", "rent", "flatrate"

  provider_names=[]
  if subtype in str(data):
    filtered_data = data[subtype]
    if len(filtered_data) > 0:
      provider_names = [item['provider_name'] for item in filtered_data]

  return provider_names


def create_movie_data(title):
  movie = find_movie(title,"en-US")
  if movie and "id" in movie:
    id = get_movie_id(movie)
    providers = get_watch_providers(id)
    flatproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"flatrate" )
    rentproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"rent" )
    buyproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"buy" )
    genres = [x['name'] for x in movie['genres']]
    prodcountries = [x['name'] for x in movie['production_countries']]
    filtered_data = {'title': movie['title'], 'flatproviders': flatproviders, 'rentproviders': rentproviders, 'buyproviders': buyproviders,
                     'adult': movie['adult'], 'genres': genres, 'budget': movie['budget'], 'overview': movie['overview'],
                     'popularity': movie['popularity'], 'poster_path':movie['poster_path'], 'release_date': movie['release_date'],
                     'runtime': movie['runtime'], 'revenue': movie['revenue'], 'vote_average':movie['vote_average'], 'vote_count': movie['vote_count'],
                      'production_countries': prodcountries
                     }
    return filtered_data

def create_basic_movie_data(title):
  movie = find_movie_basic(title,"en-US")
  if movie and "id" in movie:
    id = get_movie_id(movie)
    providers = get_watch_providers(id)
    flatproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"flatrate" )
    rentproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"rent" )
    buyproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"buy" )

    filtered_data = {'title': movie['title'], 'flatproviders': flatproviders, 'rentproviders': rentproviders, 'overview': movie['overview'],
                     'release_date': movie['release_date'],
                     }
    return filtered_data




def create_recommendation_data(item):

  ## just importent info
  filtered_data = {'title': item['title'], 'flatproviders': item['flatproviders'], 'rentproviders': item['rentproviders'], 'buyproviders': item['buyproviders']}
  ##filter only movies that are for streaming or rent available
  item =filtered_data

  if 'flatproviders' in item and len(item['flatproviders']) > 0:
      if 'rentproviders' in item and len(item['rentproviders']) > 0:
           return {"title":item['title'], 'flatproviders': item['flatproviders'], 'rentproviders': item['rentproviders']}
      else:
          return {"title":item['title'],'flatproviders': item['flatproviders']}
  elif 'rentproviders' in item and len(item['rentproviders']) > 0:
     return {"title":item['title'],'rentproviders': item['rentproviders']}

def get_movie_form_search(data:dict, search:str):
  try:
      titles = [x["title"] for x in data]
      title = closeMatches(titles, search)
      title = title[0]
      movie = [x for x in data if x['title'] == title]
      return movie[0]
  except:
    return None

# Function to find all close matches of
# input string in given list of possible strings
def closeMatches(patterns, word):
     return get_close_matches(word, patterns)

def create_data_list(fulldata):
  movieinfoasstring=""
  for movie_info in fulldata:
     formatted_info = f"Title: {movie_info['title']}, "
     formatted_info += f"Overview: {movie_info['overview']}, "
     formatted_info += f"flatproviders: {', '.join(movie_info['flatproviders'])}, "
     formatted_info += f"rentproviders: {', '.join(movie_info['rentproviders'])}\n"
     movieinfoasstring+=formatted_info
  return movieinfoasstring

In [3]:
#helpers
def _get_movie_data(titles: str):
  print(f"_get_movie_data titles input: ", titles)
  inputlist = titles.split(",")
  inputlist = [x.strip(' ') for x in inputlist]
  blacklists.append(inputlist)
  result = get_basic_data_from_tmdb_for_titles(inputlist)
  #print(result)
  return result

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from operator import itemgetter

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=openai_api_key,
    max_tokens=1000
)




# NOT USED ATM START

In [ ]:
## with history


memory = ConversationBufferMemory(return_messages=True)

recommedation_titels_prompt_history = ChatPromptTemplate.from_messages(
  [
    ("system",
    """
    You are a assistent for movie recommendations. Your goal is to recommend movies based on the users input. Recommend no movie that is allready mentioned in history. Everytime recommend 5 movie titles.
    Return a commar seperated list of english movie titles. Movies should just be strings of characters in lowercase, no number or date or parenthesis!
    """),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
  ]
)


recro_prompt_with_hitory = ChatPromptTemplate.from_messages(
  [
    ("system",
    """
    You are a assistent for movie recommendations. Your goal is to recommend movies based on the context.
    Write down your recommendation like a buddy would do. One sentence that descripes the movie for each title from context. Behind the descriptions write the flatproviders and rentproviders from context.
    context: {context}
    """),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
  ]
)


recommendation_chain = (
    {"context": RunnablePassthrough.assign(history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")) | recommedation_titels_prompt_history | llm | StrOutputParser() | RunnableLambda(_get_movie_data)}
    | recro_prompt
    | llm
    | StrOutputParser()
)

memory.save_context({"input":input}, {"output": response})
memory.load_memory_variables({})


# NOT USED ATM END

In [6]:
# Prompts
from langchain.prompts import PromptTemplate

descisionprompt = PromptTemplate.from_template(
        """ You are a assistent for everyything about movies.
        Given the user input below, classify it as either being for a Movie 'Recommendation' request, or is it a 'question' about a specific movie.

Do not respond with more than one word.

<input>
{input}
</input>

Classification:"""
    )


recommedation_titels_prompt = PromptTemplate.from_template(
        """ You are a assistent for movie recommendations. Your goal is to recommend movies based on the users input. Everytime recommend 5 movies.
            Return a commar seperated list of the english movie titles.

input: {input}
Answer:"""
)



general_prompt =  PromptTemplate.from_template(
        """Always Say that you are not sure what the user wants to know. Add that you are here for helping to find a good movie that fits the users needs.
        Answer in same language like the input.

inputQuestion: {input}
Answer:"""
    )

recro_prompt = PromptTemplate.from_template("""
    You are a assistent for movie recommendations. Your goal is to recommend movies based on the context. If context is empty: Appologize yourself kindly that you dont cant help rightnow.

    Please make sure you complete the objective above with the following rules:
    1. Answer in the same language as the input:{input} is given
    2. Act like a good friend that wants that you watch one of this movies!
    2. Write a List for every movie recommendation based on the context in the following format:

    <movietitle>: <short description in one sentence>. ( Free Streaming on: <flatproviders>, For Rent: <rentproviders>   < Behind the descriptions write the flatproviders and rentproviders from context.

    context: {context}
""")




question_answering_prompt = PromptTemplate.from_template("""
    You are a assistent for movie informations. Your goal is to anwer questions about a specific movie.
    Answer the question based on the data from the context. Answer in same language like the question.
    question: {input}
    context: {context}
""")

extract_title_prompt = PromptTemplate.from_template("""
    You are an expert for movies. Extract the movie title from the user input: {input}
    Respond just the title.
""")



In [7]:
#define chains
descissionchain = ( descisionprompt | llm | StrOutputParser() )
#recommendation_chain = (recommedation_titels_prompt | llm )

recommendation_chain = (
    {"context": recommedation_titels_prompt | llm | StrOutputParser() | RunnableLambda(_get_movie_data), "input":RunnablePassthrough()}
    #{"context": recommedation_titels_prompt | llm | StrOutputParser() | RunnableLambda(_get_movie_data)}
    | recro_prompt
    | llm
    | StrOutputParser()
)

info_chain = (
    {"context": extract_title_prompt | llm |StrOutputParser() | RunnableLambda(get_detail_data_from_tmdb_for_title), "input":RunnablePassthrough()}
    | question_answering_prompt
    | llm
    | StrOutputParser())


general_chain = (general_prompt | llm)

In [8]:
# stick everything together
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
    (lambda x: "recommendation" in x["topic"].lower(), recommendation_chain),
    (lambda x: "question" in x["topic"].lower(), info_chain),
    general_chain,
)
full_chain = {"topic": descissionchain, "input": lambda x: x["input"]} | branch




In [11]:
blacklist = ""
input = "Ich möchte Filme über die Mondlandung sehen"
full_chain.invoke({"input": input +" "+blacklist})

_get_movie_data titles input:  Apollo 13, First Man, Hidden Figures, The Right Stuff, Moon

get_basic_data_from_tmdb_for_titles ['Apollo 13', 'First Man', 'Hidden Figures', 'The Right Stuff', 'Moon\n']
get_movie_data_from_tmdb title= Apollo 13
find_movie for Apollo+13 response:  {'page': 1, 'results': [{'adult': False, 'backdrop_path': '/32zua2oKjn2EaRk7am3qK8zAlEj.jpg', 'genre_ids': [18, 36], 'id': 568, 'original_language': 'en', 'original_title': 'Apollo 13', 'overview': 'The true story of technical troubles that scuttle the Apollo 13 lunar mission in 1970, risking the lives of astronaut Jim Lovell and his crew, with the failed journey turning into a thrilling saga of heroism. Drifting more than 200,000 miles from Earth, the astronauts work furiously with the ground crew to avert tragedy.', 'popularity': 26.12, 'poster_path': '/oYUZHYMwNKnE1ef4WE5Hw2a9OAY.jpg', 'release_date': '1995-06-30', 'title': 'Apollo 13', 'video': False, 'vote_average': 7.447, 'vote_count': 5040}, {'adult': Fa

'Ich habe einige Filmempfehlungen über die Mondlandung für dich:\n\n1. Apollo 13: Die wahre Geschichte der technischen Probleme, die die Apollo 13-Mondmission im Jahr 1970 gefährden und das Leben des Astronauten Jim Lovell und seiner Crew riskieren. Der fehlgeschlagene Flug entwickelt sich zu einer aufregenden Heldengeschichte. (Kostenlos streamen auf: Amazon Prime Video, WOW, Sky Go. Zum Ausleihen: Apple TV, Amazon Video, Google Play Movies, YouTube, Sky Store, Rakuten TV, maxdome Store, MagentaTV)\n\n2. First Man: Ein Blick auf das Leben des Astronauten Neil Armstrong und die legendäre Weltraummission, die ihn am 20. Juli 1969 zum ersten Mann auf dem Mond werden ließ. (Zum Ausleihen: Apple TV, Amazon Video, Google Play Movies, YouTube, Sky Store, Rakuten TV, maxdome Store, MagentaTV)\n\n3. Hidden Figures: Die unerzählte Geschichte von Katherine G. Johnson, Dorothy Vaughan und Mary Jackson - brillante afroamerikanische Frauen, die bei der NASA arbeiten und die treibende Kraft hinter e

In [9]:
input = "Kannst du mir noch andere maritime Filme empfehlen?"
blacklist = "aber keinen von den folgenden filmens: captain phillips, master and commander: the far side of the world, the perfect storm, the abyss, the hunt for red october"
full_chain.invoke({"input": input +" "+blacklist})

_get_movie_data titles input:  The Life Aquatic with Steve Zissou, Jaws, The Poseidon Adventure, Titanic, The Deep
get_basic_data_from_tmdb_for_titles ['The Life Aquatic with Steve Zissou', 'Jaws', 'The Poseidon Adventure', 'Titanic', 'The Deep']
get_movie_data_from_tmdb title= The Life Aquatic with Steve Zissou
find_movie for The+Life+Aquatic+with+Steve+Zissou response:  {'page': 1, 'results': [{'adult': False, 'backdrop_path': '/7tWktnLQ81G8pHUkhuTebcTkcB3.jpg', 'genre_ids': [12, 35, 18], 'id': 421, 'original_language': 'en', 'original_title': 'The Life Aquatic with Steve Zissou', 'overview': "Renowned oceanographer Steve Zissou has sworn vengeance upon the rare shark that devoured a member of his crew. In addition to his regular team, he is joined on his boat by Ned, a man who believes Zissou to be his father, and Jane, a journalist pregnant by a married man. They travel the sea, all too often running into pirates and, perhaps more traumatically, various figures from Zissou's past, 

'Hier sind einige maritime Filme, die du vielleicht magst:\n\n1. The Life Aquatic with Steve Zissou: Ein renommierter Meeresforscher schwört Rache an einem seltenen Hai, der ein Mitglied seiner Crew gefressen hat. (Kostenloses Streaming auf Disney Plus, Zum Mieten: Apple TV, Amazon Video, Google Play Movies, YouTube, Sky Store, Rakuten TV, maxdome Store, MagentaTV, Microsoft Store)\n\n2. Jaws: Ein unaufhaltsamer großer weißer Hai terrorisiert die Bewohner von Amity Island, und der Polizeichef, ein Meeresforscher und ein abgebrühter Haifänger versuchen, das blutrünstige Biest zu zerstören. (Zum Mieten: Apple TV, Amazon Video, Google Play Movies, YouTube, Sky Store, Rakuten TV, maxdome Store, MagentaTV, Microsoft Store)\n\n3. The Poseidon Adventure: Als ihr Ozeandampfer kentert, kämpft eine Gruppe von Passagieren ums Überleben und um ihre Flucht. (Zum Mieten: Amazon Video, maxdome Store)\n\n4. Titanic: Eine 101-jährige Frau namens Rose DeWitt Bukater erzählt die Geschichte ihres Lebens a

In [11]:
input="wie wurde life of pi bewertet"
full_chain.invoke({"input": input +" "+blacklist})

get_movie_data_from_tmdb title= Life of Pi
find_movie for Life+of+Pi response:  {'page': 1, 'results': [{'adult': False, 'backdrop_path': '/lcyippCCbZOliGxX7HbTNyWf98q.jpg', 'genre_ids': [12, 18], 'id': 87827, 'original_language': 'en', 'original_title': 'Life of Pi', 'overview': "The story of an Indian boy named Pi, a zookeeper's son who finds himself in the company of a hyena, zebra, orangutan, and a Bengal tiger after a shipwreck sets them adrift in the Pacific Ocean.", 'popularity': 41.201, 'poster_path': '/iLgRu4hhSr6V1uManX6ukDriiSc.jpg', 'release_date': '2012-11-20', 'title': 'Life of Pi', 'video': False, 'vote_average': 7.396, 'vote_count': 12493}, {'adult': False, 'backdrop_path': '/A5RprRPOB1EQghDVoQoFWvWaYl7.jpg', 'genre_ids': [18, 10749], 'id': 62390, 'original_language': 'ja', 'original_title': 'カケラ', 'overview': "The story of the relationship between a college student whose relationship with her boyfriend is going nowhere and a bisexual medical artist who makes prosthetic

'Life of Pi wurde mit einer Bewertung von 7.396 bewertet.'

# CodeSnippets


In [1]:
import re

text = """
Apollo 13: Ein Routineflug im All wird zur gefährlichen Rettungsmission, als die Astronauten mit einer funktionsuntüchtigen Raumkapsel kämpfen müssen. ( Free Streaming on: Amazon Prime Video, WOW, Sky Go, For Rent: Apple TV, Amazon Video, Google Play Movies, YouTube, Sky Store, Rakuten TV, maxdome Store, MagentaTV)

Interstellar: Eine Gruppe von Astronauten begibt sich auf eine Reise ins Ungewisse, um einen bewohnbaren Planeten zu finden und die Menschheit zu retten. ( Free Streaming on: Netflix, Netflix basic with Ads, For Rent: Apple TV, Amazon Video, Google Play Movies, YouTube, Sky Store, Rakuten TV, maxdome Store, MagentaTV, Microsoft Store, Pantaflix)
"""

pattern = re.compile(r'^(.*?):\s', re.MULTILINE)
matches = re.findall(pattern, text)

for match in matches:
    print(match)



Apollo 13
Interstellar


In [12]:


exampledic = [{'id': 'x has a', 'x': 'a', 'y': 'c'}, {'id': 'b in y', 'x': 'u', 'y': 'b'}, {'id': 'nix', 'x': 'c', 'y': 'c'}, {'id': 'x has b', 'x': 'b', 'y': 'c'}]

values=["a","b"]

#filtered_list = [d for d in test if ('x' in d and d['x'] in ['a', 'b']) or ('y' in d and d['y'] in ['a', 'b'])]

filtered_list = [d for d in exampledic if ('x' in d and d['x'] in values) or ('y' in d and d['y'] in values)]

#filtered_list = [d for d in exampledic if ('x' in d and any(val in d['x'] for val in values)) or ('y' in d and any(val in d['y'] for val in values))]


filtered_list


[{'id': 'x has a', 'x': 'a', 'y': 'c'},
 {'id': 'b in y', 'x': 'u', 'y': 'b'},
 {'id': 'x has b', 'x': 'b', 'y': 'c'}]

In [26]:
import re

text = """Hallo Filmfreund! Klar, ich habe ein paar amüsante Filmempfehlungen für dich, die dich sicherlich gut unterhalten werden. Los geht's:

1. Guardians of the Galaxy:

Ein Abenteurer, ein sprechender Waschbär und ein Baumwesen retten das Universum. (Frei verfügbar auf: Disney Plus, zum Mieten: Apple TV, Amazon Video, Google Play Movies, YouTube, Sky Store, Rakuten TV, maxdome Store, MagentaTV, Microsoft Store, Pantaflix)

2. Super 8:

Teenager drehen einen Film und werden unfreiwillig Zeugen eines gruseligen Vorfalls. (Frei verfügbar auf: Joyn Plus, zum Mieten: Apple TV, Amazon Video, Google Play Movies, YouTube, Sky Store, Rakuten TV, maxdome Store, MagentaTV, Chili, Microsoft Store)

3. Zootopia 2:

Die Fortsetzung des animierten Films Zootopia. (Leider ist dieser Film derzeit nicht kostenfrei verfügbar, aber möglicherweise zum Mieten.)

4. Guardians of the Galaxy Vol. 2:

Die Hüter müssen kämpfen, um ihre neu gefundene Familie zusammenzuhalten und die Geheimnisse der wahren Abstammung von Peter Quill zu entwirren. (Frei verfügbar auf: Disney Plus, zum Mieten: Apple TV, Amazon Video, Google Play Movies, YouTube, Sky Store, maxdome Store, MagentaTV, Microsoft Store, Pantaflix)

Ich hoffe, diese Empfehlungen treffen deinen Geschmack! Viel Spaß beim Filmabend und möge das Popcorn immer frisch sein!
"""

print(len(text.split("("))>1)

pattern = r'\d+\.\s([^:]+)'

matches = re.findall(pattern, text)

for match in matches:
    print(match)


True
Guardians of the Galaxy
Super 8
Zootopia 2
Guardians of the Galaxy Vol. 2
